In [1]:
import nltk
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import csv
from scipy import stats
import re


from nltk.stem.snowball import SnowballStemmer
import re

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jhmak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jhmak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("./0825finaldata.csv",encoding='cp949')
# data[["Exception",'Reason','FoundFrame']]

In [3]:
# Null값이 있는지 없는지 찾아준다 - True이면 Null값이 있는것
data['Exception'].isnull().values.any()

True

In [4]:
data['Reason'].isnull().values.any()

True

In [5]:
data['FoundFrame'].isnull().values.any()

False

In [6]:
# fillna()는 Null값을 바꾸어 주는 함수/ 텍스트는 공백, 숫자는 0으로 바꾸어주는게 국룰
data['Exception'].fillna(" ", inplace=True)

In [7]:
data['Reason'].fillna(" ", inplace=True)

In [8]:
data['FoundFrame'].fillna(" ", inplace=True)

In [9]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
snowball=SnowballStemmer("english")

def normalize_document(doc):
#     doc = re.sub('<.*?>', '', doc)   # remove HTML tags
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', ' ', doc, re.I|re.A)
    doc = re.sub("\S*\d\S*", " ", doc).strip()
    doc = doc.lower()
    doc = doc.strip()
#     doc = doc.isalpha()
    
    # tokenize document
    tokens = wpt.tokenize(doc)
    
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
#     # stem words
#     stem_words = [snowball.stem(x) for x in filtered_tokens]
    
#     stem_words = [item for item in stem_words if item.isalpha()]
    
    # re-create document from filtered tokens
#     doc = ' '.join(stem_words)
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [10]:
data['Exception'] = normalize_corpus(data['Exception'])
data['Exception'].head()

0                           java lang runtimeexception
1    java lang nullpointerexception java lang nullp...
2    java lang runtimeexception java lang runtimeex...
3        java security cert certpathvalidatorexception
4                       java lang nullpointerexception
Name: Exception, dtype: object

In [11]:
data['Reason'] = normalize_corpus(data['Reason'])
data['Reason'].head()

0    test failed setlocationrelativeto operates inc...
1                                                     
2    java lang runtimeexception error initializing ...
3                       java lang nullpointerexception
4                                                     
Name: Reason, dtype: object

In [12]:
data['FoundFrame'] = normalize_corpus(data['FoundFrame'])
data['FoundFrame'].head()

0    locationrelativetotest checklocation locationr...
1    com sun prism dtexture getcontext dtexture jav...
2    com sun javafx tk toolkit gettoolkit toolkit j...
3    sun security provider certpath pkixmastercertp...
4    java lang system getproperty java base system ...
Name: FoundFrame, dtype: object

In [13]:
data.shape

(7275, 6)

In [14]:
data2 = data.dropna()
data2 = pd.DataFrame(data2)
data2.shape

(7273, 6)

In [15]:
data = data2
data = data.reset_index(drop=True)

In [29]:
data.to_csv("0825finalNLTKdata.csv", header = True, index = False)

In [16]:
data.tail()

,Exception,Reason,FoundFrame,Number Of Post,Answer Count,Priority
7268,java lang illegalargumentexception,index bounds,org eclipse swt swt error swt java org eclipse...,20,22,3.0
7269,java lang illegalargumentexception,,org eclipse core internal runtime listenerlist...,20,22,1.0
7270,java lang illegalargumentexception,,org eclipse core internal runtime listenerlist...,20,22,2.0
7271,java lang illegalargumentexception,,org eclipse core internal runtime listenerlist...,20,22,3.0
7272,java lang nullpointerexception,,org eclipse debug internal ui views console pr...,20,41,3.0


In [17]:

scaler = StandardScaler()
NOP = scaler.fit(np.array(data['Number Of Post']).reshape(-1, 1))
# NOP = pd.DataFrame(stats.zscore(np.array(data['Number Of Post']), axis=0))

NOP = pd.DataFrame(scaler.transform(np.array(data['Number Of Post']).reshape(-1, 1)))


In [18]:

AC = scaler.fit(np.array(data['Answer Count']).reshape(-1, 1))
AC = pd.DataFrame(scaler.transform(np.array(data['Answer Count']).reshape(-1, 1)))


In [19]:

exception = TfidfVectorizer().fit(data['Exception'].values.astype('U'))
exception = exception.transform(data['Exception'].values.astype('U')).toarray()
# exception.transform(data['Exception'][0:1].values.astype('U')).toarray()



In [20]:

reason = TfidfVectorizer().fit(data['Reason'].values.astype('U'))
reason = reason.transform(data['Reason'].values.astype('U')).toarray()
# reason.transform(data['Reason'][0:1].values.astype('U')).toarray()


In [21]:
foundframe = TfidfVectorizer().fit(data['FoundFrame'].values.astype('U'))
foundframe = foundframe.transform(data['FoundFrame'].values.astype('U')).toarray()
# foundframe.transform(data['FoundFrame'][0:1].values.astype('U')).toarray()

In [22]:
# reason.get_feature_names()
# foundframe.get_feature_names()
# exception.get_feature_names()

In [23]:

exception = pd.DataFrame(exception)
reason = pd.DataFrame(reason)
foundframe = pd.DataFrame(foundframe)
priority = pd.DataFrame(data['Priority'])


In [24]:

AddData1 = pd.merge(exception,reason, how= 'outer', left_index=True, right_index =True)
AddData2 = pd.merge(AddData1,foundframe, how= 'outer', left_index=True, right_index =True)
AddData3 = pd.merge(AddData2,NOP, how= 'outer', left_index=True, right_index =True)
AddData4 = pd.merge(AddData3,AC, how= 'outer', left_index=True, right_index =True)
AddData5 = pd.merge(AddData4,priority, how= 'outer', left_index=True, right_index =True)


In [25]:
AddData1.tail()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221
7268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
AddData5.to_csv("0825finalpredata.csv", header = True, index = False)

In [28]:
AddData5

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,18505,18506,18507,18508,18509,18510,18511,0_y,0,Priority
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.201486,0.477486,3.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.714565,-0.241127,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.630616,3.398302,3.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.201486,-0.171584,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.201486,-0.612024,4.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.201486,0.477486,3.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.201486,-0.612024,2.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.155683,-0.078860,4.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.201486,-0.194765,4.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.201486,0.755659,4.0


In [28]:
predata = AddData5

In [29]:

import csv
import pandas as pd
from imblearn.over_sampling import BorderlineSMOTE
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, accuracy_score


In [30]:

# X = predata.iloc[:,0:19398].values
# X1 = predata.iloc[:,19398:19400].values
#  input data 전체
X2 = predata.iloc[:,0:22323].values

Y = predata['Priority']
X2


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581,  0.47748626],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.71456489, -0.24112721],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.63061558,  3.3983023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581, -0.47293801],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581, -0.47293801],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.20148581, -0.0324975 ]])

In [31]:

X_train, X_test, y_train, y_test = train_test_split(X2, Y, test_size=0.3, random_state=0)


In [32]:

smote = BorderlineSMOTE(random_state=42)
new_X_train,new_y_train = smote.fit_resample(X_train,y_train)
print(new_y_train.value_counts())


2.0    2592
5.0    2592
3.0    2592
4.0    2592
1.0    2592
Name: Priority, dtype: int64


RandomForest 적용

In [34]:

print('random forest')

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100, random_state = 42)

#Train the model using the training sets y_pred=clf.predict(X_test)
#clf.fit(new_X_train,new_y_train)
clf.fit(new_X_train,new_y_train)

y_pred=clf.predict(X_test)

# RandomForest 정확도
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


random forest
Accuracy: 0.7506874427131073


SVM 적용

In [ ]:

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

KNN 적용 ( k = 5 )

In [ ]:

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


KNN 적용 ( k = 7 )

In [ ]:

#Create KNN Classifier
knn2 = KNeighborsClassifier(n_neighbors=7)

#Train the model using the training sets
knn2.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = knn2.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Decision Tree 적용

In [ ]:

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(new_X_train,new_y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


XGBoost 적용

In [ ]:

import xgboost as xgb
xgb_clf = xgb.XGBClassifier()

# XGBoost 예측모형
xgb_model = xgb.XGBClassifier(silent=False, 
                              verbosity=0,
                              booster='gbtree',
                              scale_pos_weight=1,
                              learning_rate=0.01,  
                              colsample_bytree = 0.4,
                              subsample = 0.8,
                              objective='binary:logistic', 
                              n_estimators=100, 
                              max_depth=5, 
                              gamma=10, 
                              seed=777)

xgb_model.fit(new_X_train, new_y_train)

y_pred = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred))


XGBoost 적용 - depth[ 3, 4, 7, 9, 10 ]에 따른 정확도 예측

In [ ]:

import time
import progressbar

accuracy_list = []

# max_depth_list = [3,5,7,9,10]
max_depth_list = [3,4,7,9,10]

bar = progressbar.ProgressBar(maxval=10).start()
for max_depth in max_depth_list:
    xgb_model = xgb.XGBClassifier(max_depth=max_depth, seed=777)
    xgb_model.fit(new_X_train,new_y_train)
    y_pred = xgb_model.predict(X_test)
    xgb_accuracy = accuracy_score(y_test, y_pred) 
    accuracy_list.append(xgb_accuracy)
    bar.update(max_depth)
    
bar.finish()
xgb_df = pd.DataFrame({'tree depth':max_depth_list, 'accuracy':accuracy_list})
xgb_df.head()    
